<a href="https://colab.research.google.com/github/tonychung30/HW3_minmax/blob/main/UPDATEDcodeformax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install  pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 25.3 MB/s eta 0:00:00


In [ ]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [ ]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # other option: LpMinimize

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [ ]:
# SuperVit
SV = pp.LpVariable(name="SuperVit",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # when would 'Integer' be needed?

# NewHealth
NH = pp.LpVariable(name="NewHealth",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients_:

In [ ]:
SVCoeff=0.2
NHCoeff=0.3
obj_func = SVCoeff * SV + NHCoeff * NH

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [ ]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Vitimin_C_constraint',   # Modified constraint name
                    e= 20*SV + 30*NH,
                    rhs=60,
                    sense=pp.LpConstraintGE)
C2= pp.LpConstraint(name='Calcium_constraint', # Modified constraint name
                    e= 500*SV + 250*NH,
                    rhs=1000,
                    sense=pp.LpConstraintGE)
C3= pp.LpConstraint(name='Iron_constraint', # Modified constraint name
                    e= 9*SV + 2*NH,
                    rhs=18,
                    sense=pp.LpConstraintLE, )
C4= pp.LpConstraint(name='Niacin_constraint', # Modified constraint name
                    e= 2*SV + 10*NH,
                    rhs=20,
                    sense=pp.LpConstraintLE, )
C5= pp.LpConstraint(name='Magnesium_constraint', # Modified constraint name
                    e= 60*SV + 90*NH,
                    rhs=360,
                    sense=pp.LpConstraintLE, )

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [ ]:
model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

/usr/local/lib/python3.11/dist-packages/pulp/pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


6. **Solve the MODEL**: Notice we are not using the _default solver_:

In [ ]:
model.solve();

7. **Basic Report**:

From above, you can print:

In [ ]:
"Model Status",pp.LpStatus[model.status]

('Model Status', 'Optimal')

Also, get these key results:

In [ ]:
Results={"Optimal Solution for Cheaper Pill":pp.value(model.objective)}
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Optimal Solution for Cheaper Pill': 0.8279069800000001,
 'NewHealth': 1.6744186,
 'SuperVit': 1.627907}

In [ ]:
#or
import pandas as pd
pd.DataFrame.from_dict(Results,orient='index',columns=['info']).map('{:,.2f}'.format)

,info
Optimal Solution for Cheaper Pill,0.83
NewHealth,1.67
SuperVit,1.63


From above, you know how much to produce (Gas and Oil) to get the most revenue subject to the constraints given.

7. **The Sensitivity of the Result** can be obtained like this:

In [ ]:
sensitivityValues = [{'constraints':name, 'shadow price':c.pi, 'slack': c.slack}
                     for name, c in model.constraints.items()]

pd.DataFrame(sensitivityValues).set_index('constraints').map('{:,.2f}'.format)

,shadow price,slack
constraints,,
Vitimin_C,-0.00,-22.79
Calcium,-0.00,-232.56
Iron,-0.00,-0.00
Niacin,-0.00,-0.00
Magnesium,-0.00,111.63
Vitimin_C_constraint,-0.00,-22.79
Calcium_constraint,-0.00,-232.56
Iron_constraint,0.02,-0.00
Niacin_constraint,0.03,-0.00
